In [ ]:
### Import libraries

import General_f as Gf
import pandas as pd
pd.set_option("display.precision", 2)
import numpy as np
import importlib
import wx
import DQ_measure_f as DQf
from matplotlib import pyplot as plt
from IPython.display import display
from matplotlib import pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn' --- Para quitar los warnings de pandas y python


### Step 1 - Characterization of Database Type

In [ ]:
importlib.reload(Gf)

app = wx.App()
dbType = Gf.MyDataBase()
dbType.CenterOnScreen()
app.SetTopWindow(dbType)
app.MainLoop()
app.Destroy()

if not dbType.ban: 
    print('You must select an option before the next step.')

### Step 2 - Characterization of Variable Type

In [ ]:
if not dbType.ban:
    print('You must select an option in the step 1.')
else:
    importlib.reload(Gf)
    df,_ = Gf.check_df()

In [ ]:
importlib.reload(Gf)

app = wx.App()
Answers = Gf.MyDims()
Answers.CenterOnScreen()
app.SetTopWindow(Answers)
app.MainLoop()
app.Destroy()

if not Answers.ban: 
    print('You must answer the questions before the next step.')

### Step 3 - Dimension Selection and Weight Assignment

In [ ]:
importlib.reload(Gf)

if not Answers.ban:
    print('You must answer the questions in the step 2.')
else:
    dims = list(set(dbType.dimensions + Answers.dimensions))

    app = wx.App()
    frame = Gf.MyFrame(dims)
    frame.CenterOnScreen()
    app.SetTopWindow(frame)
    app.MainLoop()
    app.Destroy()

### Step 4 - Evaluation of Selected Dimensions

In [ ]:
importlib.reload(Gf)
if any(x in frame.dimensions.keys() for x in ['Accuracy', 'Uncertainty', 'Concordance']):
    app = wx.App()
    file = Gf.FileUpload()
    file.CenterOnScreen()
    app.SetTopWindow(file)
    app.MainLoop()
    app.Destroy()

    if file.ban:
        ref,_ = Gf.check_df()
    else: 
        print('You must load a reference dataset.')


In [ ]:
importlib.reload(DQf)
datos = df.copy()
DQ_dim = {}

for d in frame.dimensions.items():
    if d[1] > 0:
        if d[0] == 'Completeness':
            compl, muestra = DQf.completeness(datos, clean=False)
            if not pd.isna(compl):
                DQ_dim['Completeness'] = compl
        
        #if d[0] == 'Consistency':
        #    cons,noCons,muestra = DQf.consistency(datos, clean=False)
        #    if not pd.isna(cons):
        #        DQ_dim['Consistency'] = cons

        if d[0] == 'Uniqueness':
            uniq, _, muestra = DQf.uniqueness(datos, clean=False)
            if not pd.isna(uniq):
                DQ_dim['Uniqueness'] = uniq

        if d[0] == 'Redundancy':
            red,_,muestra = DQf.redundancy(datos, clean=False)
            if not pd.isna(red):
                DQ_dim['Redundancy'] = red

        #if d[0] == 'Volume':
        #    vol,muestra = DQf.volume(datos, clean=False)
        #    if not pd.isna(vol):
        #        DQ_dim['Volume'] = vol

        if d[0] == 'Precision':
            prec,muestra = DQf.precision(datos)
            if not pd.isna(prec):
                DQ_dim['Precision'] = prec

        #if d[0] == 'Outliers':
        #    out,muestra = DQf.outliers(datos)
        #    if not pd.isna(out):
        #        DQ_dim['Outliers'] = out   

        if d[0] == 'Uncertainty':
            unc,muestra = DQf.uncertainty(datos, ref)
            if not pd.isna(unc):
                DQ_dim['Uncertainty'] = unc
        
        if d[0] == 'Accuracy':
            acc,muestra = DQf.accuracy(datos, ref)
            if not pd.isna(acc):
                DQ_dim['Accuracy'] = acc
        
        if d[0] == 'Concordance':
            conc,muestra = DQf.concordance(datos, ref)
            if not pd.isna(conc):
                DQ_dim['Concordance'] = conc



### Step 5 - DQIndex Calculation

In [ ]:


DQ_Index = 0
for d in DQ_dim.keys():
    DQ_Index += round((frame.dimensions[d]/sum(frame.dimensions.values())*DQ_dim[d]),2)
print(DQ_dim)
print('DQ_index:',round(DQ_Index,2))


dims = list(DQ_dim.keys())
dims = [*dims, dims[0]]

scores = list(DQ_dim.values())
scores = [*scores, scores[0]]

label_loc = np.linspace(start=0, stop=2 * np.pi, num=len(dims))

plt.figure(figsize=(6,6))
plt.subplot(polar=True)
plt.plot(label_loc, scores, 'o-')
plt.title('Dimensions', size=20)
plt.thetagrids(np.degrees(label_loc), labels=dims)
plt.fill(label_loc, scores, alpha=0.25)
plt.yticks([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
plt.ylim(0,1)
#plt.savefig('ideal_lvl1.svg', format='svg')
plt.show()